In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import random

In [2]:
def bagDt():
    trfname=input("Write your train data file name : ")
    tstfname=input("Write your test data file name : ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','


    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        header='infer'
    elif hd=='2':
        header=None


    cl=input("Select column# where your y vlaues exist: ")
    yno = int(cl)-1

    train = pd.read_csv(trfname,sep=form,header=header)
    test = pd.read_csv(tstfname,sep=form,header=header)

    yCol=train.columns[yno]

    catloc = input("input categorical value location (ex. 1, 2, 3, 9) 1 means first column and you shuld seperate column wiht comma(,): ")
    catLocInt = list(map(lambda x : int(x)-1 ,catloc.split(',')))

    for i in catLocInt +[yno]:
        train.iloc[:,i] = pd.Categorical(train.iloc[:,i])
        test.iloc[:,i] = pd.Categorical(test.iloc[:,i])

    X = train.drop(yCol,1)
    Y = train.iloc[:, yno]
    df = pd.concat([X, Y], axis=1)

    tX = test.drop(yCol,1)
    tY = test.iloc[:, yno]
    tdf = pd.concat([tX, tY], axis=1)

    bagTrainLst=[]
    random.seed(0)
    for i in range(51):
        bagIndex = []
        for i in range(train.shape[0]):
            bagIndex.append(random.randint(0,train.shape[0]-1))
        bagTrain = train.iloc[bagIndex,:]
        bagTrain.index = range(bagTrain.shape[0])
        bagTrainLst.append(bagTrain)

    dt2lv = DecisionTreeClassifier(random_state=0, max_depth=2)
    dt4lv = DecisionTreeClassifier(random_state=0, max_depth=4)


    def getyhat2lv(bagTrain):
            bagY=bagTrain.iloc[:,yno]
            bagX=bagTrain.drop(yCol,1)
            dt2lv.fit(bagX,bagY)
            return(dt2lv.predict(tX))

    def getyhat4lv(bagTrain):
            bagY=bagTrain.iloc[:,yno]
            bagX=bagTrain.drop(yCol,1)
            dt4lv.fit(bagX,bagY)
            return(dt2lv.predict(tX))

    yhLst2lv = []
    for bgTr in bagTrainLst:
        yhLst2lv.append(getyhat2lv(bgTr))

    bagYhat2lv = pd.DataFrame(yhLst2lv).mode().values 

    cont2lv = pd.crosstab(tY,list(bagYhat2lv))
    acc2lv = sum(np.diag(cont2lv))/sum(sum(cont2lv.values))


    yhLst4lv = []
    for bgTr in bagTrainLst:
        yhLst4lv.append(getyhat4lv(bgTr))

    bagYhat4lv = pd.DataFrame(yhLst4lv).mode().values 

    cont4lv = pd.crosstab(tY,list(bagYhat4lv))
    acc4lv = sum(np.diag(cont4lv))/sum(sum(cont4lv.values))

    #print term
    cont2lv.columns.name = 'Predicted Class'
    cont2lv.index.name = 'Actual Class'

    cont4lv.columns.name = 'Predicted Class'
    cont4lv.index.name = 'Actual Class'

    predy2lv = pd.DataFrame([range(1,(len(test)+1)),tY,bagYhat2lv[0]]).transpose()
    predy2lv.columns=['ID','Actual class','tree-depth2 pred']

    predy4lv = pd.DataFrame([range(1,(len(test)+1)),tY,bagYhat4lv[0]]).transpose()
    predy4lv.columns=['ID','Actual class','tree-depth4 pred']

    fname = input('Write your output file name : ')

    print(predy2lv.to_string(index=False))
    print(cont2lv)
    print("Overall accuracy = %0.3f" %(acc2lv))

    print(predy4lv.to_string(index=False))
    print(cont4lv)
    print("Overall accuracy = %0.3f" %(acc4lv))

    out = open(fname, 'w')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print("(1) Tree with depth 2",file=out)   
        print(predy2lv.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (tree-depth2)",file=out)
        print(cont2lv,file=out)
        print("",file=out)
        print("Model Summary (tree-depth2)",file=out)
        print("Overall accuracy = %0.3f" %(acc2lv),file=out)
        print("",file=out)

        print("(2) Tree with depth 4",file=out)
        print(predy4lv.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (tree-depth4)",file=out)
        print(cont4lv,file=out)
        print("",file=out)
        print("Model Summary (tree-depth4)",file=out)
        print("Overall accuracy = %0.3f" %(acc4lv),file=out)

    out.close()

In [9]:
bagDt()

Write your train data file name : pid.dat
Write your test data file name : pidtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Select the data header format(1 = with header or 2 = no header): 2
Select column# where your y vlaues exist: 8
input categorical value location (ex. 1, 2, 3, 9) 1 means first column and you shuld seperate column wiht comma(,): 1
Write your output file name : out11.txt
ID  Actual class  tree-depth2 pred
  1             1                 2
  2             1                 1
  3             1                 2
  4             1                 1
  5             1                 2
  6             1                 2
  7             1                 2
  8             1                 2
  9             1                 1
 10             1                 2
 11             1                 1
 12             1                 2
 13             1                 2
 14             1                 2
 15             1                 1
 16     